# Taller Teoria de Codigo (Reduccion, perforación, sindrome)

### Integrantes: Kenny Zhu y Yovany Zhu

In [796]:
from itertools import product
import itertools
import numpy as np

## Funciones

In [797]:
def create_copy_matrix(matrix):
    new_matrix = []
    for row in matrix:
        copy_row = []
        for element in row:
            copy_row.append(element)
        new_matrix.append(copy_row)
    return new_matrix

def generate_code(matrix, q, _):
    code = generate_combination(q, len(matrix))
    codeword = [[sum(a * b for a, b in zip(row, col)) % q for col in zip(*matrix)] for row in code]
    return codeword

def generate_combination(q, n):
    code = []
    for i in range(q):
        code.append(i)

    combinations = [list(comb) for comb in product(code, repeat=n)]

    return combinations

def hamming_distance(codeword1, codeword2):
    return sum(el1 != el2 for el1, el2 in zip(codeword1, codeword2))

def find_minimum_distance(codewords):
    min_dist = None 
    for i in range(len(codewords)):
        for j in range(i + 1, len(codewords)):
            dist = hamming_distance(codewords[i], codewords[j])
            if min_dist is None or dist < min_dist:
                min_dist = dist
    return min_dist

def matrix_control(generator_matrix):
    k, n = len(generator_matrix), len(generator_matrix[0])
    identity_part = [row[:k] for row in generator_matrix]
    parity_part = [row[k:] for row in generator_matrix]
    parity_transpose = [list(x) for x in zip(*parity_part)]
    identity_for_H = np.eye(n - k, dtype=int).tolist()
    H_control = [parity_transpose[i] + identity_for_H[i] for i in range(n - k)]
    for i in range(len(H_control)):
        for j in range(len(H_control[0])):
            H_control[i][j] = H_control[i][j] % 2
    return H_control

def parameters(codeword):
    n = len(codeword[0])
    k = len(codeword)
    d = find_minimum_distance(codeword)
    return n, k, d

## Codigo Extension

In [798]:
def generate_extension_code(code, q, n):
    for i in range(len(code)):
        count = 0
        for j in range(n):
            count += code[i][j]
        if count % q == 0:
            code[i].insert(n + 1, 0)
        else:
            code[i].insert(n + 1, q - count % q)
    return code


## Codigo reducción

In [799]:
def reduction(code, i):
    code = create_copy_matrix(code)
    codeword = []
    for n in range(len(code)):
        if code[n][i] == 0:
            codeword.append(code[n])
            del codeword[len(codeword) - 1][i]
    return codeword

## Codigo de perforación

In [800]:
def perforation(code, i):
    code = create_copy_matrix(code)
    codeword = []
    for n in code:
        new_codeword = n[:i] + n[i+1:]
        codeword.append(new_codeword)
    return codeword

## Sindrome

In [801]:
def vector_with_least_ones(codes):
    min_ones = float('inf')
    vector_least_ones = None

    for code in codes:
        count_ones = sum(code)

        if count_ones < min_ones:
            min_ones = count_ones
            vector_least_ones = code

    return vector_least_ones


def generate_all_codewords(length):
    return list(itertools.product([0, 1], repeat=length))


def find_missing_codewords(all_codewords, existing_codewords):
    missing_codewords = []
    for codeword in all_codewords:
        if list(codeword) not in existing_codewords:
            missing_codewords.append(codeword)
    code = list(vector_with_least_ones(missing_codewords))
    return code


def find_side_classes(matrix, q, n):
    code = generate_code(matrix, q, n)
    error = []
    error.append(vector_with_least_ones(code))
    codes = create_copy_matrix(code)
    print(f"error 1: {vector_with_least_ones(code)} : {codes}")
    all_possible_codewords = generate_all_codewords(n)

    for i in range(q**(n - len(matrix)) - 1):
        missing_codewords = find_missing_codewords(all_possible_codewords, codes)
        new_codes = [[(x + y) % 2 for x, y in zip(code, missing_codewords)] for code in code]
        print(f"error {i + 2}: {missing_codewords} : {new_codes}")
        codes.extend(new_codes)
        error.append(missing_codewords)

    return error

## Solución Taller

1. (20 points) Considere el código C sobre $ \mathbb{F}_3 $ con matriz generadora $ G $ y responda las preguntas a continuación.

   $$ G = \left[ \begin{array}{cccccc}
   2 & 1 & 0 & 0 & 1 & 1 \\
   1 & 0 & 2 & 2 & 1 & 0 \\
   0 & 1 & 0 & 0 & 2 & 1 \\
   \end{array} \right] $$

   a. Halle el código de extensión $ \hat{C} $.


In [802]:
matrix = [[2, 1, 0, 0, 1, 1],[1, 0, 2, 2, 1, 0],[0, 1, 0, 0, 2, 1]]

code = generate_code(matrix, 3, 6)

extension_code = generate_extension_code(code, 3, 6)

extension_code

[[0, 0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 2, 1, 2],
 [0, 2, 0, 0, 1, 2, 1],
 [1, 0, 2, 2, 1, 0, 0],
 [1, 1, 2, 2, 0, 1, 2],
 [1, 2, 2, 2, 2, 2, 1],
 [2, 0, 1, 1, 2, 0, 0],
 [2, 1, 1, 1, 1, 1, 2],
 [2, 2, 1, 1, 0, 2, 1],
 [2, 1, 0, 0, 1, 1, 1],
 [2, 2, 0, 0, 0, 2, 0],
 [2, 0, 0, 0, 2, 0, 2],
 [0, 1, 2, 2, 2, 1, 1],
 [0, 2, 2, 2, 1, 2, 0],
 [0, 0, 2, 2, 0, 0, 2],
 [1, 1, 1, 1, 0, 1, 1],
 [1, 2, 1, 1, 2, 2, 0],
 [1, 0, 1, 1, 1, 0, 2],
 [1, 2, 0, 0, 2, 2, 2],
 [1, 0, 0, 0, 1, 0, 1],
 [1, 1, 0, 0, 0, 1, 0],
 [2, 2, 2, 2, 0, 2, 2],
 [2, 0, 2, 2, 2, 0, 1],
 [2, 1, 2, 2, 1, 1, 0],
 [0, 2, 1, 1, 1, 2, 2],
 [0, 0, 1, 1, 0, 0, 1],
 [0, 1, 1, 1, 2, 1, 0]]

b. Halle los parámetros de $ \hat{C} $.

In [803]:
parametros = parameters(extension_code)
print(f"Los parametros son: \n n = {parametros[0]}, k = {parametros[1]}, d = {parametros[2]}")

Los parametros son: 
 n = 7, k = 27, d = 3


c. Halle $ \hat{C_i} $, para $ 1 \leq i \leq 6 $.

In [804]:
codeword = generate_code(matrix, 3, 6)

codeword

[[0, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 2, 1],
 [0, 2, 0, 0, 1, 2],
 [1, 0, 2, 2, 1, 0],
 [1, 1, 2, 2, 0, 1],
 [1, 2, 2, 2, 2, 2],
 [2, 0, 1, 1, 2, 0],
 [2, 1, 1, 1, 1, 1],
 [2, 2, 1, 1, 0, 2],
 [2, 1, 0, 0, 1, 1],
 [2, 2, 0, 0, 0, 2],
 [2, 0, 0, 0, 2, 0],
 [0, 1, 2, 2, 2, 1],
 [0, 2, 2, 2, 1, 2],
 [0, 0, 2, 2, 0, 0],
 [1, 1, 1, 1, 0, 1],
 [1, 2, 1, 1, 2, 2],
 [1, 0, 1, 1, 1, 0],
 [1, 2, 0, 0, 2, 2],
 [1, 0, 0, 0, 1, 0],
 [1, 1, 0, 0, 0, 1],
 [2, 2, 2, 2, 0, 2],
 [2, 0, 2, 2, 2, 0],
 [2, 1, 2, 2, 1, 1],
 [0, 2, 1, 1, 1, 2],
 [0, 0, 1, 1, 0, 0],
 [0, 1, 1, 1, 2, 1]]

In [805]:
for i in range(6):
    print(f"perforación {i+1}: {perforation(codeword, i)}")

perforación 1: [[0, 0, 0, 0, 0], [1, 0, 0, 2, 1], [2, 0, 0, 1, 2], [0, 2, 2, 1, 0], [1, 2, 2, 0, 1], [2, 2, 2, 2, 2], [0, 1, 1, 2, 0], [1, 1, 1, 1, 1], [2, 1, 1, 0, 2], [1, 0, 0, 1, 1], [2, 0, 0, 0, 2], [0, 0, 0, 2, 0], [1, 2, 2, 2, 1], [2, 2, 2, 1, 2], [0, 2, 2, 0, 0], [1, 1, 1, 0, 1], [2, 1, 1, 2, 2], [0, 1, 1, 1, 0], [2, 0, 0, 2, 2], [0, 0, 0, 1, 0], [1, 0, 0, 0, 1], [2, 2, 2, 0, 2], [0, 2, 2, 2, 0], [1, 2, 2, 1, 1], [2, 1, 1, 1, 2], [0, 1, 1, 0, 0], [1, 1, 1, 2, 1]]
perforación 2: [[0, 0, 0, 0, 0], [0, 0, 0, 2, 1], [0, 0, 0, 1, 2], [1, 2, 2, 1, 0], [1, 2, 2, 0, 1], [1, 2, 2, 2, 2], [2, 1, 1, 2, 0], [2, 1, 1, 1, 1], [2, 1, 1, 0, 2], [2, 0, 0, 1, 1], [2, 0, 0, 0, 2], [2, 0, 0, 2, 0], [0, 2, 2, 2, 1], [0, 2, 2, 1, 2], [0, 2, 2, 0, 0], [1, 1, 1, 0, 1], [1, 1, 1, 2, 2], [1, 1, 1, 1, 0], [1, 0, 0, 2, 2], [1, 0, 0, 1, 0], [1, 0, 0, 0, 1], [2, 2, 2, 0, 2], [2, 2, 2, 2, 0], [2, 2, 2, 1, 1], [0, 1, 1, 1, 2], [0, 1, 1, 0, 0], [0, 1, 1, 2, 1]]
perforación 3: [[0, 0, 0, 0, 0], [0, 1, 0, 2, 1], 

d. Halle $ \hat{C_i} $, para $ 1 \leq i \leq 6 $.

In [806]:
for i in range(6):
    print(f"reducción {i+1}: {reduction(codeword, i)}")

reducción 1: [[0, 0, 0, 0, 0], [1, 0, 0, 2, 1], [2, 0, 0, 1, 2], [1, 2, 2, 2, 1], [2, 2, 2, 1, 2], [0, 2, 2, 0, 0], [2, 1, 1, 1, 2], [0, 1, 1, 0, 0], [1, 1, 1, 2, 1]]
reducción 2: [[0, 0, 0, 0, 0], [1, 2, 2, 1, 0], [2, 1, 1, 2, 0], [2, 0, 0, 2, 0], [0, 2, 2, 0, 0], [1, 1, 1, 1, 0], [1, 0, 0, 1, 0], [2, 2, 2, 2, 0], [0, 1, 1, 0, 0]]
reducción 3: [[0, 0, 0, 0, 0], [0, 1, 0, 2, 1], [0, 2, 0, 1, 2], [2, 1, 0, 1, 1], [2, 2, 0, 0, 2], [2, 0, 0, 2, 0], [1, 2, 0, 2, 2], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]
reducción 4: [[0, 0, 0, 0, 0], [0, 1, 0, 2, 1], [0, 2, 0, 1, 2], [2, 1, 0, 1, 1], [2, 2, 0, 0, 2], [2, 0, 0, 2, 0], [1, 2, 0, 2, 2], [1, 0, 0, 1, 0], [1, 1, 0, 0, 1]]
reducción 5: [[0, 0, 0, 0, 0], [1, 1, 2, 2, 1], [2, 2, 1, 1, 2], [2, 2, 0, 0, 2], [0, 0, 2, 2, 0], [1, 1, 1, 1, 1], [1, 1, 0, 0, 1], [2, 2, 2, 2, 2], [0, 0, 1, 1, 0]]
reducción 6: [[0, 0, 0, 0, 0], [1, 0, 2, 2, 1], [2, 0, 1, 1, 2], [2, 0, 0, 0, 2], [0, 0, 2, 2, 0], [1, 0, 1, 1, 1], [1, 0, 0, 0, 1], [2, 0, 2, 2, 2], [0, 0, 1, 1, 0]

2. (30 points) Considere el código $ C $ sobre $ F_2 $ con matriz generadora $ G $ y responda las preguntas a continuación.

$$ G = \begin{pmatrix}
1 & 0 & 0 & 1 & 0 & 1 & 0 \\
0 & 1 & 0 & 1 & 1 & 0 & 1 \\
0 & 0 & 1 & 1 & 0 & 0 & 1
\end{pmatrix} $$

a. Halle el código de extensión $ \hat{C} $.\
b. Halle los parámetros de $ \hat{C} $.\
c. Halle $ \hat{c}_i $, para $ 1 \leq i \leq 3 $.\
d. Halle $ \hat{C}_i $, para $ 1 \leq i \leq 3 $.\
e. Halle una matriz de control $ H $ para el código $ C $.\
f. Decodifique el vector 1110011 usando síndromes.


a.

In [807]:
matrix = [[1, 0, 0, 1, 0, 1, 0],[0, 1, 0, 1, 1, 0, 1],[0, 0, 1, 1, 0, 0, 1]]

code = generate_code(matrix, 2, 7)

extension_code = generate_extension_code(code, 2,7)

extension_code

[[0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 1, 1],
 [0, 1, 0, 1, 1, 0, 1, 0],
 [0, 1, 1, 0, 1, 0, 0, 1],
 [1, 0, 0, 1, 0, 1, 0, 1],
 [1, 0, 1, 0, 0, 1, 1, 0],
 [1, 1, 0, 0, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0, 0]]

b.

In [808]:
parametros = parameters(extension_code)
print(f"Los parametros son: \n n = {parametros[0]}, k = {parametros[1]}, d = {parametros[2]}")

Los parametros son: 
 n = 8, k = 8, d = 4


c.

In [809]:
codeword = generate_code(matrix, 2, 7)

codeword

[[0, 0, 0, 0, 0, 0, 0],
 [0, 0, 1, 1, 0, 0, 1],
 [0, 1, 0, 1, 1, 0, 1],
 [0, 1, 1, 0, 1, 0, 0],
 [1, 0, 0, 1, 0, 1, 0],
 [1, 0, 1, 0, 0, 1, 1],
 [1, 1, 0, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 0]]

In [810]:
for i in range(3):
    print(f"perforación {i+1}: {perforation(codeword, i)}")

perforación 1: [[0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1], [1, 0, 1, 1, 0, 1], [1, 1, 0, 1, 0, 0], [0, 0, 1, 0, 1, 0], [0, 1, 0, 0, 1, 1], [1, 0, 0, 1, 1, 1], [1, 1, 1, 1, 1, 0]]
perforación 2: [[0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1], [0, 0, 1, 1, 0, 1], [0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 1, 1], [1, 0, 0, 1, 1, 1], [1, 1, 1, 1, 1, 0]]
perforación 3: [[0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 1], [0, 1, 1, 1, 0, 1], [0, 1, 0, 1, 0, 0], [1, 0, 1, 0, 1, 0], [1, 0, 0, 0, 1, 1], [1, 1, 0, 1, 1, 1], [1, 1, 1, 1, 1, 0]]


d.

In [811]:
for i in range(3):
    print(f"reducción {i+1}: {reduction(codeword, i)}")

reducción 1: [[0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1], [1, 0, 1, 1, 0, 1], [1, 1, 0, 1, 0, 0]]
reducción 2: [[0, 0, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1], [1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 1, 1]]
reducción 3: [[0, 0, 0, 0, 0, 0], [0, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0], [1, 1, 0, 1, 1, 1]]


e.

In [812]:
H_matrix = matrix_control(matrix)
print("La matriz de control es:")
for row in H_matrix:
    print(row)

La matriz de control es:
[1, 1, 1, 1, 0, 0, 0]
[0, 1, 0, 0, 1, 0, 0]
[1, 0, 0, 0, 0, 1, 0]
[0, 1, 1, 0, 0, 0, 1]


f.

In [813]:
errors = [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 1],
    [0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 1, 1, 0],
    [0, 0, 0, 1, 1, 0, 0],
    [0, 0, 1, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1, 0],
    [1, 0, 0, 0, 1, 0, 0],
    [1, 1, 0, 0, 0, 0, 0]
]

# Multiplicación de matrices
resultados = []
for sublist in errors:
    resultado = [(sum(a * b for a, b in zip(row, sublist))) % 2 for row in H_matrix]
    resultados.append(resultado)

i = 0
for n in resultados:
    print(f"Error: {errors[i]} Sindrome: {n}")
    i += 1

Error: [0, 0, 0, 0, 0, 0, 0] Sindrome: [0, 0, 0, 0]
Error: [0, 0, 0, 0, 0, 0, 1] Sindrome: [0, 0, 0, 1]
Error: [0, 0, 0, 0, 0, 1, 0] Sindrome: [0, 0, 1, 0]
Error: [0, 0, 0, 0, 1, 0, 0] Sindrome: [0, 1, 0, 0]
Error: [0, 0, 0, 1, 0, 0, 0] Sindrome: [1, 0, 0, 0]
Error: [0, 0, 1, 0, 0, 0, 0] Sindrome: [1, 0, 0, 1]
Error: [0, 1, 0, 0, 0, 0, 0] Sindrome: [1, 1, 0, 1]
Error: [1, 0, 0, 0, 0, 0, 0] Sindrome: [1, 0, 1, 0]
Error: [0, 0, 0, 0, 0, 1, 1] Sindrome: [0, 0, 1, 1]
Error: [0, 0, 0, 0, 1, 0, 1] Sindrome: [0, 1, 0, 1]
Error: [0, 0, 0, 0, 1, 1, 0] Sindrome: [0, 1, 1, 0]
Error: [0, 0, 0, 1, 1, 0, 0] Sindrome: [1, 1, 0, 0]
Error: [0, 0, 1, 0, 0, 1, 0] Sindrome: [1, 0, 1, 1]
Error: [0, 1, 0, 0, 0, 1, 0] Sindrome: [1, 1, 1, 1]
Error: [1, 0, 0, 0, 1, 0, 0] Sindrome: [1, 1, 1, 0]
Error: [1, 1, 0, 0, 0, 0, 0] Sindrome: [0, 1, 1, 1]


In [814]:
matrix = [
    [1, 1, 1, 1, 0, 0, 0],
    [0, 1, 0, 0, 1, 0, 0],
    [1, 0, 0, 0, 0, 1, 0],
    [0, 1, 1, 0, 0, 0, 1]
]
vector = [1, 1, 1, 0, 0, 1, 1]

result = [sum(matrix_row[i] * vector[i] for i in range(len(vector))) % 2 for matrix_row in matrix]

print(f'H.v = {result}')


H.v = [1, 1, 0, 1]


In [815]:
error = [0, 1, 0, 0, 0, 0, 0]
code = [1, 1, 1, 0, 0, 1, 1]

sum_result = [((a + b) % 2) for a, b in zip(error, code)]

print("Palabra decodificada =", sum_result)

Palabra decodificada = [1, 0, 1, 0, 0, 1, 1]
